In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

from src.transform.transform import data_transforms
from src.data.dataset import HCDDataset
from src.model.HCD_resnet50 import Resnet50
from src.train import train_model

%matplotlib inline
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

print (torch.__version__)

0.4.1.post2


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print (device)
root_path = '/home/kwu/data/kaggle/HCD'
train_csv_path = os.path.join(root_path, 'train_labels.csv')
train_data_path = os.path.join(root_path, 'train')
test_data_path = os.path.join(root_path, 'test')

cuda:0


In [3]:
def load_data():
    
    df = pd.read_csv(train_csv_path)
    
    data_df = {}
    data_df['train'], data_df['val'] = train_test_split(
        df,
        test_size=0.1,
        random_state=42,
        shuffle=True,
    )

    datasets = {
        x : HCDDataset(
            root=root_path,
            in_df=data_df[x],
            transform=data_transforms[x],
        )
        for x in ['train', 'val']
    }

    dataloaders = {
        x:DataLoader(
            dataset=datasets[x],
            batch_size=128,
            shuffle=True,
            num_workers=4
        )
        for x in ['train', 'val']
    }

    datasets_sizes = {x : len(datasets[x]) for x in ['train', 'val']}
    
    return dataloaders, datasets_sizes

In [4]:
dataloaders, datasets_sizes = load_data()

model_ft = Resnet50(2)

model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(
    filter(lambda p: p.requires_grad, model_ft.parameters()),
    lr=0.001,
    momentum=0.9
)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

model_ft = train_model(
    model=model_ft,
    device=device,
    dataloaders=dataloaders,
    datasets_sizes=datasets_sizes,
    criterion=criterion,
    optimizer=optimizer_ft,
    scheduler=exp_lr_scheduler,
    num_epoches=10
)


Epoch train 1 :   0%|          | 0/198144 [00:00<?, ?it/s]

train  Acc: 0.9157, val Acc: 0.9456


Epoch train 2 :   0%|          | 0/198144 [00:00<?, ?it/s]

train  Acc: 0.9500, val Acc: 0.9566


Epoch train 3 :   0%|          | 0/198144 [00:00<?, ?it/s]

...

train  Acc: 0.9845, val Acc: 0.9733


Epoch train 9 :   0%|          | 0/198144 [00:00<?, ?it/s]

train  Acc: 0.9856, val Acc: 0.9735


Epoch val 9 : 100%|██████████| 22016/22016 [00:10<00:00, 2197.10it/s, loss=0.09065]


train  Acc: 0.9869, val Acc: 0.9705
Training complete in 43m 59s
Best val ACC :0.973458


In [7]:
torch.save(model_ft.state_dict(),'2018-11-20-0.973458.pt')